In [9]:
from functions_collection_copy import *
import time
import statistics
import csv

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5
seqs=['aaatt',
 'cagtcgtt',
 'agctg',
 'ctgtagaac',
 'gcagg',
 'tttttt',
 'agcggcg',
 'cacccgtga',
 'tctgcag',
 'taactcata']

In [5]:
data =np.array([['', '7', '0', '1'],
        ['', '5', '0', '2'],
        ['', '28', '0', '3'],
        ['', '29', '0', '4'],
        ['', '25', '0', '5'],
        ['', '34', '0', '6'],
        ['', '34', '0', '7'],
        ['', '34', '0', '8'],
        ['', '12', '1', '2'],
        ['', '35', '1', '3'],
        ['', '29', '1', '4'],
        ['', '32', '1', '5'],
        ['', '29', '1', '6'],
        ['', '29', '1', '7'],
        ['', '34', '1', '8'],
        ['', '25', '2', '3'],
        ['', '34', '2', '4'],
        ['', '28', '2', '5'],
        ['', '39', '2', '6'],
        ['', '39', '2', '7'],
        ['', '39', '2', '8'],
        ['', '10', '3', '4'],
        ['', '5', '3', '5'],
        ['', '32', '3', '6'],
        ['', '32', '3', '7'],
        ['', '35', '3', '8'],
        ['', '15', '4', '5'],
        ['', '32', '4', '6'],
        ['', '32', '4', '7'],
        ['', '27', '4', '8'],
        ['', '29', '5', '6'],
        ['', '29', '5', '7'],
        ['', '32', '5', '8'],
        ['', '0', '6', '7'],
        ['', '5', '6', '8'],
        ['', '5', '7', '8']])


edges=np.array([['',1,'1','2'],['',8,'1','3'],['',2,'1','4'],['',9,'2','3'],['',5,'3','4'],['',10,'2','4']],[])

In [17]:
def find_min_span_edges_testing(pseudomatrix, verbose=False): #the function actually implementing Kruskal's algo
    sorted_indices = np.lexsort((pseudomatrix[:, 1].astype(int),)) #sort edges to have the shorter ones first.
    E = pseudomatrix[sorted_indices]
    if verbose: 
        print("this is E (sorted matrix without any stars yet): ")
        print(E)
    names= np.unique(pseudomatrix[:, 2:]) #extracting all node names to keep track of them in name_dict.
    name_dict= {letter:i for i, letter in enumerate(names)}
    if verbose:
        print("the names of nodes going into the first name dict are: "+str(names)+" and the name_dict is orginally "+ str(name_dict))
    E[:,0]='' #always start proces by setting zero'th col as empy. May be redundant, but just to be sure ;) 
    x=0 #to keep track of current row
    it=0 # iteration number for print statement...
    while len(set(name_dict.values()))>1: #so while all nodes are still not unified in one tree..
        it+=1 #only there for print statement below :) 
        if verbose: print("\n \n this is it "+str(it)+" of the find_min_span_edges_testing-func.") #the aforementioned print statement!
        min0,min1,min2=E[x][1],E[x][2],E[x][3] #take the next row in the pseudomatrix to get the len of the edge and the names of the two nodes it connects
        tree1_id= name_dict[min1] #getting the positions of the nodes currently in question in the current list of trees
        tree2_id= name_dict[min2]
        if tree2_id == tree1_id: #if the two nodes are already in the same tree, skip this row.
            if verbose: print("the two nodes are already in the same tree")
            x += 1
            continue
        else:
            E[x][0] = "*" #if they were not already in the same tree, then mark that the row/edge is going to be used!
            #generally making sure that the new tree assigned to the node is the one with the lowest number. Also updating name_dict to keep the name interval 'closed'.
            if tree1_id < tree2_id:
                orig_tree2_id = tree2_id
                for key, value in name_dict.items():
                    if value == orig_tree2_id:
                        name_dict[key] = tree1_id
                    elif value > orig_tree2_id:
                        name_dict[key] -= 1
            else:
                orig_tree1_id = tree1_id
                for key, value in name_dict.items():
                    if value == orig_tree1_id:
                        name_dict[key] = tree2_id
                    elif value > orig_tree1_id:
                        name_dict[key] -= 1
            x += 1
            if verbose: print("after that iteration, we end up with this dict: "+ str(name_dict)+ "and the set is: "+ str(set(name_dict.values()))+" and the len of that set is "+ str(len(set(name_dict.values()))))
    if verbose: 
        print ("Here are the edges included in the MST, marked with a star! \n")
        print(E)
            
    res_mat=E 
    return res_mat

IndexError: index 2 is out of bounds for axis 1 with size 1